In [ ]:
# ------------------------------------------------------------------
# 0. 依賴套件
# pip install pandas sentence-transformers faiss-cpu tqdm
# ------------------------------------------------------------------
import re
from collections import defaultdict

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

# ------------------------------------------------------------------
# 1. 讀檔
# ------------------------------------------------------------------
df_jobs = pd.read_csv("output_skills.csv")
df_courses = pd.read_csv("coursera_skills.csv")

# 若原檔沒有唯一索引，可先臨時補 job_id / course_id
df_jobs["job_id"] = df_jobs.index
df_courses["course_id"] = df_courses.index

# ------------------------------------------------------------------
# 2. 找出「技能欄」：regex r"^技能\d+$"
# ------------------------------------------------------------------
skill_pattern = re.compile(r"^技能\d+$")

job_skill_cols = [c for c in df_jobs.columns if skill_pattern.match(c)]
course_skill_cols = [c for c in df_courses.columns if skill_pattern.match(c)]

print("Job skill cols  :", job_skill_cols)
print("Course skill cols:", course_skill_cols)

Job skill cols  : ['技能1', '技能2', '技能3', '技能4', '技能5', '技能6', '技能7', '技能8', '技能9', '技能10', '技能11', '技能12', '技能13', '技能14', '技能15', '技能16', '技能17']
Course skill cols: ['技能1', '技能2', '技能3', '技能4', '技能5', '技能6', '技能7', '技能8', '技能9', '技能10', '技能11', '技能12', '技能13', '技能14', '技能15', '技能16', '技能17', '技能18', '技能19']


In [ ]:
# ------------------------------------------------------------------
# 3. 寬表 ➜ 長表
# ------------------------------------------------------------------
job_long = df_jobs.melt(
    id_vars=["job_id"],
    value_vars=job_skill_cols,
    value_name="skill",
).dropna(subset=["skill"])

course_long = df_courses.melt(
    id_vars=["course_id"],
    value_vars=course_skill_cols,
    value_name="skill",
).dropna(subset=["skill"])


# ------------------------------------------------------------------
# 4. 技能前處理（去空白、全小寫，可自行擴充同義詞替換）
# ------------------------------------------------------------------
def clean(text: str) -> str:
    return str(text).strip().lower()


job_long["skill"] = job_long["skill"].map(clean)
course_long["skill"] = course_long["skill"].map(clean)

# ------------------------------------------------------------------
# 5. 產生課程技能向量，建立 FAISS 索引
# ------------------------------------------------------------------
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

unique_course_skills = course_long["skill"].unique()
course_vecs = model.encode(unique_course_skills, normalize_embeddings=True)

dim = course_vecs.shape[1]
index = faiss.IndexFlatIP(dim)  # 小型資料直接暴力搜尋即可
index.add(course_vecs)

# (skill_id → 原文字) 與 (skill_id → course_id list) 對照
skillid2skill = {i: s for i, s in enumerate(unique_course_skills)}
skillid2courses = defaultdict(list)
for row in course_long.itertuples():
    sid = np.where(unique_course_skills == row.skill)[0][0]
    skillid2courses[sid].append(row.course_id)


# ------------------------------------------------------------------
# 6. 建一個技能→課程快取 (向量檢索版)
# ------------------------------------------------------------------
def courses_for_skill(skill_text, top_k=10, thr=0.6):
    v = model.encode([skill_text], normalize_embeddings=True)
    sims, idxs = index.search(v, top_k)
    matched = []
    for sim, idx in zip(sims[0], idxs[0]):
        if sim < thr:
            break
        matched += skillid2courses[idx]
    return matched


skill2courses = {sk: courses_for_skill(sk) for sk in job_long["skill"].unique()}


# ------------------------------------------------------------------
# 7. Greedy Set-Cover：一次覆蓋所有技能
# ------------------------------------------------------------------
def recommend_for_job(job_id: int):
    req_skills = job_long.loc[job_long.job_id == job_id, "skill"].unique()
    skills_left = set(req_skills)
    chosen_courses, coverage = [], defaultdict(list)

    while skills_left:
        best_course, best_gain = None, 0
        candidate_courses = {
            cid for sk in skills_left for cid in skill2courses.get(sk, [])
        }
        for cid in candidate_courses:
            gain = sum(cid in skill2courses.get(sk, []) for sk in skills_left)
            if gain > best_gain:
                best_gain, best_course = gain, cid
        if best_course is None:
            break  # 資料缺口
        chosen_courses.append(best_course)
        for sk in list(skills_left):
            if best_course in skill2courses.get(sk, []):
                skills_left.remove(sk)
                coverage[best_course].append(sk)
    return chosen_courses, coverage, skills_left


# ------------------------------------------------------------------
# 8. Demo：隨便挑一筆職缺
# ------------------------------------------------------------------
def pretty_report(job_id: int):
    courses, explain, missing = recommend_for_job(job_id)

    # ❶ 需要學習的技能
    need_skills = job_long.loc[job_long.job_id == job_id, "skill"].unique()
    print(f"\n* 需要學習的技能：{', '.join(need_skills)}")

    # ❷ 推薦課程列表
    print("* 推薦課程如下：")
    for idx, cid in enumerate(courses, 1):
        row = df_courses.loc[df_courses.course_id == cid].iloc[0]
        cname = row["課程名稱"]
        curl = row.get("課程網址", "（無網址欄位）")
        sk_list = "、".join(explain[cid])
        print(f"  課程{idx}：{cname}  課程連結：{curl}  對應技能：{sk_list}")

    # ❸ 尚無對應課程的技能
    if missing:
        print("\n⚠️ 以下技能目前找不到對應課程：", "、".join(missing))


# ------------------------------------------------------------------
# 9. 呼叫報表（範例：第一筆職缺）
# ------------------------------------------------------------------
pretty_report(df_jobs.iloc[2]["job_id"])


* 需要學習的技能：node.js, javascript es6, typescript, mongodb, redis, docker, distributed services, http, https, websocket, tcp/ip, devops, aws, gcp, digital ocean, github, git
* 推薦課程如下：
  課程1：Java FullStack Developer  課程連結：https://www.coursera.org/specializations/java-fullstack  對應技能：devops、http、javascript es6、mongodb、git、redis、aws、websocket
  課程2：Microsoft Full-Stack Developer  課程連結：https://www.coursera.org/professional-certificates/microsoft-full-stack-developer  對應技能：github、https
  課程3：Full Stack Web Development en Español  課程連結：https://www.coursera.org/specializations/fullstack-web-development-espanol  對應技能：typescript、node.js
  課程4：Google IT Support  課程連結：https://www.coursera.org/professional-certificates/google-it-support  對應技能：tcp/ip
  課程5：Blockchain Basics  課程連結：https://www.coursera.org/learn/blockchain-basics  對應技能：distributed services
  課程6：Meta Android Developer  課程連結：https://www.coursera.org/professional-certificates/meta-android-developer  對應技能：docker

⚠️ 以下技能目前找不到對應課程： gcp、digi